# <center><font color='#F1B03D'>**Revenue Intelligence Enhancement for BrokerChooser - Data Processing**</font></center>
### <center><font color='#F1B03D'>Central European University, 2024-2025</font></center>
### <center><font color='#F1B03D'>CEU Capstone Project</font></center>

### <left><font color='#F1B03D'>Author: Péter Bence Török (torokpe@gmail.com)</font></left>
### <left><font color='#F1B03D'>BrokerChooser Contact Person: Zoltán Molnár (zoltan.molnar@brokerchooser.com)</font></left>

---
<p style="font-size:22px;"> The notebook acts as a full data processing pipeline that prepares raw session-level data for analysis. It includes cleaning missing or inconsistent values, formatting variables, grouping rare categories, and engineering new features. As a final step, it outputs two cleaned datasets in .csv format: one containing the full processed data and another with a 10% random sample, both ready for modeling and analysis.

In [ ]:
# Importing necessary libraries
import os

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Merging all raw .csv files
folder_path = "/Path/to/file/location/Raw files"

csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 1. Data cleaning and formatting

In [ ]:
# Remove duplicate rows based on session_id, keeping only the first occurrence
merged_df.drop_duplicates(subset="session_id", keep="first", inplace=True)

# Replace missing values in the revenue_usd column with 0 (assuming no revenue was generated)
merged_df["revenue_usd"] = merged_df["revenue_usd"].fillna(0)

# Replace missing values in page_referrer and aggregated_brokers columns with the string "None"
merged_df[["page_referrer", "aggregated_brokers"]] = merged_df[["page_referrer", "aggregated_brokers"]].fillna("None")

# Drop all rows that have any missing values left in any column
merged_df.dropna(subset=merged_df.columns, inplace=True)

# Filter out test sessions by removing rows where 'aggregated_pages' contains 'brokerchooser.test'
merged_df = merged_df[~merged_df["aggregated_pages"].astype(str).str.contains("brokerchooser.test", na=False)]

In [ ]:
# Define the list of operating systems want to keep as individual categories
valid_os = ["Windows", "iOS", "Android", "Macintosh", "Chrome OS"]

# Replace all other, less common operating systems with the label "other"
merged_df["op_system"] = merged_df["op_system"].apply(
    lambda x: x if x in valid_os else "other"
)

In [ ]:
# Define the list of browser types want to keep as individual categories
valid_browsers = ["Chrome", "Safari", "Edge", "Firefox", "Opera"]

# Replace all other, less common browser types with the label "other"
merged_df["browser"] = merged_df["browser"].apply(
    lambda x: x if x in valid_browsers else "other"
)

In [ ]:
# Simplify traffic source names by keeping main categories and grouping all others as "other"
merged_df["traffic_name"] = merged_df["traffic_name"].apply(
    lambda x: x if x in ["(referral)", "(organic)", "(direct)"] else "other"
)

In [ ]:
# Replace '(none)' traffic medium values with 'other' for better category grouping
merged_df['traffic_medium'] = merged_df['traffic_medium'].replace('(none)', 'other')

In [ ]:
# Count how many sessions are from each country
country_counts = (
    merged_df
    .groupby("country")
    .size()
    .reset_index(name="count")
    .sort_values("count", ascending=False)
)

# Select the top 50 countries based on session count
top_countries = set(country_counts.iloc[:50]["country"])

# Group less frequent countries under a single "other" category
merged_df["country"] = merged_df["country"].apply(
    lambda x: x if x in top_countries else "other"
)

In [ ]:
# Simplify device column by keeping main categories and grouping all others as "other"
merged_df["device"] = merged_df["device"].apply(
    lambda x: x if x in ['desktop', 'mobile'] else 'other'
)

In [ ]:
# Define a function to clean strings by removing or replacing specific characters
def clean_string(val):
    if isinstance(val, str):
        return (
            val.replace(' ', '')
               .replace("’", '')
               .replace("(", '')
               .replace(")", '')
               .replace("-", '_')
               .replace("&", '')
        )
    return val  # return as-is if not a string

# Apply the cleaning function to every value in the DataFrame
merged_df = merged_df.applymap(clean_string)

# 2. Feature Engineering

In [ ]:
# Calculate session duration in seconds by subtracting start from end timestamp and converting microseconds to seconds
merged_df["session_duration_in_seconds"] = (merged_df["end_event_timestamp"] - merged_df["start_event_timestamp"]) / 1000000

In [ ]:
# Count number of visited pages by counting commas in the aggregated list and adding 1
merged_df["pages_visited_count"] = merged_df["aggregated_pages"].str.count(",") + 1

# Count number of triggered events in sessions
merged_df["events_count"] = merged_df["aggregated_event_names"].str.count(",") + 1

# Count how many different broker pages were visited during the session
merged_df["brokers_visited_count"] = merged_df["aggregated_brokers"].str.count(",") + 1

In [ ]:
# Convert event_date column to datetime type
merged_df["event_date"] = pd.to_datetime(merged_df["event_date"])

# Extract day of week (0=Monday, 6=Sunday)
merged_df["day_of_week"] = merged_df["event_date"].dt.dayofweek

# Extract day of the month (1–31)
merged_df["day_of_month"] = merged_df["event_date"].dt.day

# Convert microseconds to datetime
merged_df["start_event_datetime"] = pd.to_datetime(merged_df["start_event_timestamp"], unit="us")

# Extract hour (24h format)
merged_df["start_event_hour"] = merged_df["start_event_datetime"].dt.hour
merged_df.drop(columns=["start_event_datetime"], inplace=True)

merged_df["is_weekend"] = merged_df["day_of_week"].isin([5, 6]).astype(int)

In [ ]:
# Create a binary variable to indicate whether the session generated any revenue
merged_df["generated_revenue"] = (merged_df["revenue_usd"] != 0).astype(int)

In [ ]:
# Define keywords to check for in the 'aggregated_pages' column, mapping them to new binary feature names
url_keywords = {
    "visited_about_us": "/about-us",
    "visited_reviews": "/broker-review",
    "visited_safety": "/safety/",
    "visited_best_brokers": "/best-brokers",
    "visited_scam_shield": "/scam-broker-shield",
    "visited_community_site": "community.brokerchooser",
    "visited_education": "/education/",
    "visited_find_my_broker": "/find-my-broker",
    "visited_long_term_investment": "/invest-long-term",
    "visited_broker_comparing": "/compare",
    "visited_carrers": "/careers",
    "visited_Etoro": "etoro",
    "visited_Fusion_Markets": "fusion-markets",
    "visited_Eightcap": "eightcap",
    "visited_FP_Markets": "fp-markets",
    "visited_Capital_com": "capitalcom",
    "visited_Pepperstone": "pepperstone",
    "visited_Active_Trader": "interactivetrade",
    "visited_Moneta_Markets": "moneta-markets"
}

# Create a new binary column for each keyword indicating whether that keyword was found in the user's visited pages
for col_name, keyword in url_keywords.items():
    merged_df[col_name] = merged_df["aggregated_pages"].astype(str).str.contains(keyword, na=False).astype(int)

In [ ]:
# Drop unnecessary columns
merged_df.drop(columns=["session_id", "aggregated_pages", "revenue_usd", "aggregated_event_names", "aggregated_brokers", "start_event_timestamp", "end_event_timestamp", "page_referrer", "traffic_source", "continent", "op_system_version", "browser_version", "event_date"], inplace=True)

In [ ]:
# Take a random 10% sample of the dataset and save the sampled data as a CSV file
merged_df.sample(frac=0.1, random_state=42).to_csv("/Users/Peter/Desktop/Capstone_analytical/Data files/Processed files/data_sample.csv", index=False)

# Save the full, cleaned dataset to a CSV file
merged_df.to_csv("/Users/Peter/Desktop/Capstone_analytical/Data files/Processed files/processed_data.csv", index=False)